### Replacing layers in model

In [42]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
import tensorflow as tf
import tensorflow
from tensorflow.keras import layers, activations, models, datasets
from tensorflow.keras.models import clone_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [43]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data() 

In [44]:
print(test_images.shape)

(10000, 32, 32, 3)


In [45]:
base_model = VGG16(input_shape=(32,32,3), weights='imagenet', include_top=False, classes=10, pooling='avg')
head = base_model.output
head = layers.Dense(10, activation="softmax")(head)
original_model = models.Model(inputs=base_model.input, outputs=head)

In [46]:
original_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0   

In [47]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   validation_split =0.2)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow(train_images,
                                     train_labels,
                                     subset='training',
                                     batch_size=32)

val_generator = train_datagen.flow(train_images,
                                   train_labels,
                                   subset='validation',
                                   batch_size=32)

test_generator = test_datagen.flow(test_images,
                                   test_labels,
                                   batch_size=32)

In [48]:
for layer in original_model.layers:
    if hasattr(layer, 'activation'):
        layer.activation = tensorflow.keras.activations.gelu

In [49]:
for index, layer in enumerate(original_model.layers):
    print(index, layer.name)
    if hasattr(layer, "activation"):
        print(layer.activation.__name__)

0 input_5
1 block1_conv1
gelu
2 block1_conv2
gelu
3 block1_pool
4 block2_conv1
gelu
5 block2_conv2
gelu
6 block2_pool
7 block3_conv1
gelu
8 block3_conv2
gelu
9 block3_conv3
gelu
10 block3_pool
11 block4_conv1
gelu
12 block4_conv2
gelu
13 block4_conv3
gelu
14 block4_pool
15 block5_conv1
gelu
16 block5_conv2
gelu
17 block5_conv3
gelu
18 block5_pool
19 global_average_pooling2d_1
20 dense_1
gelu


In [50]:
original_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), loss='sparse_categorical_crossentropy', 
                      metrics=['accuracy'])

In [51]:
original_model.fit(train_generator,
                   steps_per_epoch=len(train_generator),
                   validation_data=val_generator,
                   validation_steps=len(val_generator),
                   epochs=10)

Epoch 1/10
1250/1250 [==============================] - 2057s 2s/step - loss: 2.3839 - accuracy: 0.9983 - val_loss: 2.3026 - val_accuracy: 1.0000
Epoch 2/10
1250/1250 [==============================] - 2046s 2s/step - loss: 2.3026 - accuracy: 1.0000 - val_loss: 2.3026 - val_accuracy: 1.0000
Epoch 3/10
1250/1250 [==============================] - 6694s 5s/step - loss: 2.3026 - accuracy: 1.0000 - val_loss: 2.3026 - val_accuracy: 1.0000
Epoch 4/10
1250/1250 [==============================] - 2092s 2s/step - loss: 2.3026 - accuracy: 1.0000 - val_loss: 2.3026 - val_accuracy: 1.0000
Epoch 5/10
1250/1250 [==============================] - 2499s 2s/step - loss: 2.3026 - accuracy: 1.0000 - val_loss: 2.3026 - val_accuracy: 1.0000
Epoch 6/10
1250/1250 [==============================] - 2066s 2s/step - loss: 2.3026 - accuracy: 1.0000 - val_loss: 2.3026 - val_accuracy: 1.0000
Epoch 7/10
1250/1250 [==============================] - 2219s 2s/step - loss: 2.3026 - accuracy: 1.0000 - val_loss: 2.3026 -

In [ ]:
original_model.evaluate(test_generator,
                        steps=len(test_generator))

### Replacing with optimal layers

In [24]:
def Find_optimal_nodes(n_layers, prev_layer_filters):
    nodes = []
    
    for i in range(n_layers):
        nodes.append(prev_layer_filters)
        prev_layer_filters = int(prev_layer_filters/2)
    
    return nodes

In [301]:
layers_dict ={}
for ind, layer in enumerate(original_model.layers):
    if ind != len(model.layers)-1:
        layers_dict[layer.name] = ind
print(layers_dict)

{'input_31': 0, 'block1_conv1': 1, 'block1_conv2': 2, 'block1_pool': 3, 'block2_conv1': 4, 'block2_conv2': 5, 'block2_pool': 6, 'block3_conv1': 7, 'block3_conv2': 8, 'block3_conv3': 9, 'block3_pool': 10, 'block4_conv1': 11, 'block4_conv2': 12, 'block4_conv3': 13, 'block4_pool': 14, 'block5_conv1': 15, 'block5_conv2': 16, 'block5_conv3': 17, 'block5_pool': 18, 'flatten': 19, 'fc1': 20, 'fc2': 21}


In [303]:
def replace_intermediate_layer(model, layer_id, new_layer):
    from keras.models import Model

    layers = [l for l in model.layers]

    x = layers[0].outputFind_
    print(x)
    for i in range(1, len(layers)):
        if i == layer_id:
            x = new_layer(x)
        else:
            x = layers[i](x)

    new_model = Model(inputs=layers[0].input, outputs=x)
    return new_model

def insert_intermediate_layer(model, layer_id, new_layer):
    from keras.models import Model

    layers = [l for l in model.layers]
    print(layer_id, model.layers[layer_id-1].name)
    x = layers[0].output
    for i in range(1, len(layers)):
        if i == layer_id:
            x = new_layer(x)
        x = layers[i](x)

    new_model = Model(inputs=layers[0].input, outputs=x)
    return new_model

In [304]:
unsupported_ops = ['gelu', 'elu', 'selu']

In [305]:
count = 0
for index,layer in enumerate(model.layers):
    if layer.name in layers_dict:
        #print(layers_dict[layer.name], layer.name)
        if hasattr(layer,"activation"):
            if layer.activation.__name__ in unsupported_ops:
                if len(layer.output_shape)==4:
                    nodes = FindLayerNodesLinear(2, layer.output_shape[3])
                    model = insert_intermediate_layer_in_keras(model, index+count+1, layers.Dense(nodes[0], activation='relu'))
                    model = insert_intermediate_layer_in_keras(model, index+count+1, layers.Dense(nodes[1], activation='relu'))
                    count += 2
                    #model.summary()
                elif len(layer.output_shape)==2:
                    nodes = FindLayerNodesLinear(2, layer.output_shape[1])
                    model = insert_intermediate_layer_in_keras(model, index+count+1, layers.Dense(nodes[0], activation='relu'))
                    count += 1
    else:
        continue

2 block1_conv1
2 block1_conv1
5 block1_conv2
5 block1_conv2
9 block2_conv1
9 block2_conv1
12 block2_conv2
12 block2_conv2
16 block3_conv1
16 block3_conv1
19 block3_conv2
19 block3_conv2
22 block3_conv3
22 block3_conv3
26 block4_conv1
26 block4_conv1
29 block4_conv2
29 block4_conv2
32 block4_conv3
32 block4_conv3
36 block5_conv1
36 block5_conv1
39 block5_conv2
39 block5_conv2
42 block5_conv3
42 block5_conv3
47 fc1
49 fc2


In [306]:
for layer in model.layers:
    if hasattr(layer, "activation"):
        if layer.activation.__name__ == 'gelu':
            delattr(layer, "activation")

In [307]:
for layer in model.layers:
    if hasattr(layer, "activation"):
        print(layer.name, layer.activation.__name__)
    else:
        print(layer.name)

input_31
block1_conv1
dense_247 relu
dense_246 relu
block1_conv2
dense_249 relu
dense_248 relu
block1_pool
block2_conv1
dense_251 relu
dense_250 relu
block2_conv2
dense_253 relu
dense_252 relu
block2_pool
block3_conv1
dense_255 relu
dense_254 relu
block3_conv2
dense_257 relu
dense_256 relu
block3_conv3
dense_259 relu
dense_258 relu
block3_pool
block4_conv1
dense_261 relu
dense_260 relu
block4_conv2
dense_263 relu
dense_262 relu
block4_conv3
dense_265 relu
dense_264 relu
block4_pool
block5_conv1
dense_267 relu
dense_266 relu
block5_conv2
dense_269 relu
dense_268 relu
block5_conv3
dense_271 relu
dense_270 relu
block5_pool
flatten
fc1
dense_272 relu
fc2
dense_273 relu
predictions


In [308]:
model.summary()

Model: "model_253"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_31 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
dense_247 (Dense)            (None, 224, 224, 32)      2080      
_________________________________________________________________
dense_246 (Dense)            (None, 224, 224, 64)      4160      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
dense_249 (Dense)            (None, 224, 224, 32)      2080      
_________________________________________________________________
dense_248 (Dense)            (None, 224, 224, 64)      41

In [ ]:
for index, layer in enumerate(model.layers):
    if hasattr(layer, "activation"):
        if layer.activation.__name__ == "relu":
            model.layers[index].trainable = True
        else:
            model.layers[index].trainable = False
    else:
        model.layers[index].trainable = False

In [ ]:
count = 0
for index,layer in enumerate(model.layers):
    if layer.name in layers_dict:
        print(layers_dict[layer.name], layer.name)
        if hasattr(layer,"activation"):
            if layer.activation.__name__ == 'gelu':
                if len(layer.output_shape)==4:
                    model = insert_intermediate_layer_in_keras(model, index+count+1, layers.Dense(layer.output_shape[3], activation='relu'))
                    count += 1
                    #model.summary()
                elif len(layer.output_shape)==2:
                    model = insert_intermediate_layer_in_keras(model, index+count+1, layers.Dense(layer.output_shape[1], activation='relu'))
                    count += 1
    else:
        continue

In [ ]:
model = replace_intermediate_layer_in_keras(model, 
print(model.summary())
model = insert_intermediate_layer_in_keras(model, 4, BatchNormalization())
print(model.summary())

In [25]:
for index,layer in enumerate(dummy_model.layers):
    print(index, dummy_model.layers[index].name)
    if index == 3:
        break
    else:
        if hasattr(dummy_model.layers[index],"activation"):
            if layer.activation.__name__ == 'gelu':
                if len(dummy_model.layers[index].output_shape)==4:
                    head = dummy_model.output
                    head = layers.Dense(dummy_model.layers[index].output_shape[3], activation='relu')(head)
                    output = layers.Dense(dummy_model.layers[index].output_shape[3], activation='relu')(head)
                    new_model = models.Model(inputs=dummy_model.input, outputs=output)
                    model = new_model
                    new_model.summary()

0 input_3
1 block1_conv1
Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None

In [84]:
new_model.summary()

Model: "model_50"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
dense_354 (Dense)            (None, 224, 224, 64)      4160      
_________________________________________________________________
dense_356 (Dense)            (None, 224, 224, 64)      4160      
_________________________________________________________________
dense_357 (Dense)            (None, 224, 224, 64)      4160      
Total params: 14,272
Trainable params: 14,272
Non-trainable params: 0
_________________________________________________________________


In [29]:
index = 0
while index != len(model.layers)-1:
    if hasattr(model.layers[index],"activation"):
        if len(model.layers[index].output_shape)==4:
            print("1")
            print(model.layers[index].name)
            head = model.layers[index].output
            head = layers.Dense(((model.layers[index].output_shape[3])/2), activation='relu')(head)
            output = layers.Dense(model.layers[index].output_shape[3], activation='relu')(head)
            model = models.Model(inputs=model.input, outputs=output) 
        elif len(model.layers[index].output_shape)==2:
            print("2")
            print(model.layers[index].name)
            head = model.layers[index].output
            head = layers.Dense(((model.layers[index].output_shape[1])/2), activation='relu')(head)
            head = layers.Dense(model.layers[index].output_shape[1], activation='relu')(head)
    index += 1
print(index, model.layers[index].name)
output = model.layers[index].output
new_model = models.Model(inputs=model.input, outputs=output) 

1
block1_conv1
1
block1_conv2
1
block2_conv1
1
block2_conv2
1
block3_conv1
1
block3_conv2
1
block3_conv3
1
block4_conv1
1
block4_conv2
1
block4_conv3
1
block5_conv1
1
block5_conv2
1
block5_conv3
2
fc1
2
fc2
22 predictions


In [22]:
for index, layer in enumerate(new_model.layers):
    print(index, layer.name)
    if hasattr(layer, 'activation'):
        print(layer.activation.__name__)

0 input_1
1 block1_conv1
relu
2 block1_conv2
relu
3 block1_pool
4 block2_conv1
relu
5 block2_conv2
relu
6 block2_pool
7 block3_conv1
relu
8 block3_conv2
relu
9 block3_conv3
relu
10 block3_pool
11 block4_conv1
relu
12 block4_conv2
relu
13 block4_conv3
relu
14 block4_pool
15 block5_conv1
relu
16 block5_conv2
relu
17 block5_conv3
relu
18 block5_pool
19 flatten
20 fc1
relu
21 fc2
relu
22 predictions
softmax


In [23]:
new_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [46]:
for layer in model.layers:
    if hasattr(layer, 'activation'):
        layer.activation = tensorflow.keras.activations.gelu

In [55]:
to_replace = ['gelu']
for index,layer in enumerate(model.layers):
    print(index, layer.name)
    if hasattr(layer, "activation"):
        if layer.activation.__name__ in to_replace: 
            print("Activation Name :", layer.activation.__name__)
            delattr(layer, "activation")
        else:
            print("No activation function to replace")
    else:
        print("No activation function present")

0 input_4
No activation function present
1 block1_conv1
No activation function present
2 block1_conv2
No activation function present
3 block1_pool
No activation function present
4 block2_conv1
No activation function present
5 block2_conv2
No activation function present
6 block2_pool
No activation function present
7 block3_conv1
No activation function present
8 block3_conv2
No activation function present
9 block3_conv3
No activation function present
10 block3_pool
No activation function present
11 block4_conv1
No activation function present
12 block4_conv2
No activation function present
13 block4_conv3
No activation function present
14 block4_pool
No activation function present
15 block5_conv1
No activation function present
16 block5_conv2
No activation function present
17 block5_conv3
No activation function present
18 block5_pool
No activation function present
19 flatten
No activation function present
20 fc1
No activation function present
21 fc2
No activation function present
22 predic

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
test_data = ImageDataGenerator(rescale=1./255)
train_data = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

train_set = train_data.flow_from_directory("/home/dell/Downloads/archive/imagenet-mini/train", 
                                           target_size=(224,224), batch_size=64, class_mode='sparse')
test_set = test_data.flow_from_directory('/home/dell/Downloads/archive/imagenet-mini/val',
                                         target_size=(224,224), batch_size=64, class_mode='sparse')

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.evaluate(test_set, steps=len(test_set))

In [ ]:
def replace_activation_function(act_func):
    return getattr(tensorflow.keras.activations, act_func)

In [ ]:
to_replace = ['gelu']
replace_with = ['tanh', 'relu']
count = 0
architectures = {}
dummy_model = clone_model(model)
for act_func in replace_with:
    count += 1
    for layer in model.layers:
        if hasattr(layer, "activation"):
            print(layer.activation.__name__)
            if layer.activation.__name__ in to_replace:
                layer.activation = replace_activation_function(act_func)
    architectures[count] = clone_model(model)
    model = dummy_model
    for layer in model.layers:
        if hasattr(layer, "activation"):
            print("Layer Name: ",layer.name,", Activation Name :", layer.activation.__name__)